## 예제 2.1 토큰화 코드

In [13]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


## 예제 2.2 토큰 아이디에서 벡터로 변환

In [3]:
import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids))
input_embeddings = input_embeddings.unsqueeze(0)
print(f"shape: {input_embeddings.shape}")
print(f"embedding: {input_embeddings}")

shape: torch.Size([1, 5, 16])
embedding: tensor([[[-0.8989, -0.7829,  1.0097,  0.6984,  0.5762, -1.1438, -0.6242,
           0.7963,  0.5647, -0.2268, -0.4924, -0.7774, -0.1097, -0.2298,
           2.1904,  0.2229],
         [ 0.0757,  1.5489, -0.6447,  0.4154,  0.6332, -0.1329, -1.2106,
          -1.3371, -0.1560, -0.2890,  1.3018, -0.2338,  1.7527,  0.0892,
           1.4164,  0.6387],
         [-1.1524, -0.3284, -0.0551,  1.0712,  1.3251,  0.2536,  0.5346,
           0.4892, -0.3636, -1.9304, -1.1637,  0.1573, -0.8734,  1.4249,
          -0.3997, -0.5594],
         [-0.9178, -1.3168, -0.4967,  1.3440,  1.6285, -2.4251,  0.1618,
           0.2290,  0.9579,  0.8979,  1.9504,  0.1880, -1.5089,  1.6096,
           0.1364,  1.1089],
         [-1.1592, -1.6617, -0.7340,  0.1359,  0.2002, -1.6263, -0.1228,
          -0.8329, -1.5891,  0.3412, -0.1175,  0.2106,  0.6662, -0.4544,
           0.4688,  2.2253]]], grad_fn=<UnsqueezeBackward0>)


## 예제 2.3 절대적 위치 인코딩

In [4]:
embedding_dim = 16
max_position = 12
# 토큰 임베딩 층 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
# 위치 인코딩 층 생성
position_embed_layer = nn.Embedding(max_position, embedding_dim)

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
print(f"position_ids: {position_ids}")
position_encodings = position_embed_layer(position_ids)
print(f"position_encodings: {position_encodings}")
token_embeddings = embed_layer(torch.tensor(input_ids))
token_embeddings = token_embeddings.unsqueeze(0)

# 토큰 임베딩과 위치 인코딩을 더해 최종 입력 임베딩 생성
input_embeddings = token_embeddings + position_encodings
print(f"input_embeddings.shape: {input_embeddings.shape}")

position_ids: tensor([[0, 1, 2, 3, 4]])
position_encodings: tensor([[[ 0.1850, -1.4910, -1.1346, -0.7657,  1.6132, -1.7511,  2.0782,
           0.0796, -1.4046,  0.4035, -2.0391, -0.1013,  0.2305, -0.6535,
           0.1593,  0.2376],
         [ 0.5238,  0.0051, -0.7478,  0.8095, -0.3439, -2.1219, -0.3824,
           0.1655,  0.0461,  0.7539, -1.1021, -0.3834, -1.1017, -0.9535,
          -0.3936,  1.3097],
         [ 0.5729, -0.8689, -0.8217,  0.6492,  2.6804, -0.6246, -0.4054,
          -0.1487,  1.4473, -0.3233, -0.2010, -0.3219, -0.0690, -1.4358,
           1.9119,  0.4605],
         [ 0.6670, -0.4039, -0.6003,  0.2912, -0.3128,  1.5148, -0.8338,
          -0.0050,  2.2244,  1.3744,  0.2087, -1.3347,  1.2319, -0.8684,
           1.4215,  0.6391],
         [-0.2436,  0.7457, -1.3710,  0.4665, -0.9259, -0.3234, -0.0589,
          -0.4910, -0.0777,  1.1461,  0.9975, -1.1443, -0.7954,  0.3772,
           0.8681, -1.4612]]], grad_fn=<EmbeddingBackward0>)
input_embeddings.shape: torch.Siz

## 예제 2.4 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

In [5]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 6)
keys = weight_k(input_embeddings) # (1, 5, 6)
values = weight_v(input_embeddings) # (1, 5, 6)

## 예제 2.5. 스케일 점곱 방식의 어텐션

In [6]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
  dim_k = querys.size(-1) # 16
  scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return weights @ values


## 예제 2.6. 어텐션 연산의 입력과 출력

In [7]:
print(f"원본 입력 형태: {input_embeddings.shape}")

after_attention_embeddings = compute_attention(querys, keys, values)

print(f"어텐션 적용 후 형태: {after_attention_embeddings.shape}")

원본 입력 형태: torch.Size([1, 5, 16])
어텐션 적용 후 형태: torch.Size([1, 5, 16])


In [8]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

## 예제 2.8. 멀티 헤드 어텐션 구현

In [9]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    self.n_head = n_head
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    B, T, C = querys.size()
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

    attention = compute_attention(querys, keys, values, is_causal=self.is_causal)
    output = attention.transpose(1, 2).contiguous().view(B, T, C)
    output = self.concat_linear(output)
    return output


n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
print(f"shape: {after_attention_embeddings.shape}")

shape: torch.Size([1, 5, 16])


## 예제 2.9. 층 정규화 코드

In [10]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
print(f"norm_x.shape: {norm_x.shape}")

print(f"norm_x.mean(dim=-1).data: {norm_x.mean(dim=-1).data}")
print(f"norm_x.std(dim=-1).data: {norm_x.std(dim=-1).data}")

norm_x.shape: torch.Size([1, 5, 16])
norm_x.mean(dim=-1).data: tensor([[0.0000e+00, 7.4506e-09, 2.2352e-08, 4.0978e-08, 1.4901e-08]])
norm_x.std(dim=-1).data: tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]])


## 예제 2.10. 피드 포워드 층 코드

In [11]:
class PreLayerNormFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
    self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃 층 1
    self.dropout2 = nn.Dropout(dropout) # 드랍아웃 층 2
    self.activation = nn.GELU() # 활성 함수
    self.norm = nn.LayerNorm(d_model) # 층 정규화

  def forward(self, src):
    x = self.norm(src)
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x

## 예제 2.11. 인코더 층


In [12]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead) # 멀티 헤드 어텐션 클래스
    self.norm1 = nn.LayerNorm(d_model) # 층 정규화
    self.dropout1 = nn.Dropout(dropout) # 드롭아웃
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드 포워드

  def forward(self, src):
    norm_x = self.norm1(src)
    attn_output = self.attn(norm_x, norm_x, norm_x)
    x = src + self.droppout1(attn_output) # 잔차 연결

    # 피드 포워드
    x = self.feed_forward(x)
    return x

## 예제 2.12. 인코더 구현

In [14]:
import copy

def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
      output = mod(output)
    return output

## 예제 2.13. 디코더에서 어텐션 연산(마스크 어텐션)

In [15]:
def compute_attention(querys, keys, values, is_causal=False):
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    dim_k = querys.size(-1) # 16
    scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1, 5, 5)
    if is_causal:
      query_length = querys.size(2)
      key_length = keys.size(2)
      temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)
      scores = scores.masked_fill(temp_mask == False, float("-inf"))
    weights = F.softmax(scores, dim=-1) # (1, 5, 5)
    return weights @ values # (1, 5, 16)

## 예제 2.14. 크로스 어텐션이 포함된 디코더 층

In [16]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    super().__init__()
    self.self_attn = MultiheadAttention(d_model, d_model, nhead)
    self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)

  def forward(self, tgt, encoder_output, is_causal=True):
    # 셀프 어텐션 연산
    x = self.norm1(tgt)
    x = x + self.dropout1(self.self_attn(x, x, x, is_causal=is_causal))
    # 크로스 어텐션 연산
    x = self.norm2(x)
    x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))
    # 피드 포워드 연산
    x = self.feed_forward(x)
    return x

## 예제 2.15. 디코더 구현

In [17]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerDecoder(nn.Module):
  def __init__(self, decoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(decoder_layer, num_layers)
    self.num_layers = num_layers

  def forward(self, tgt, src):
    output = tgt
    for mod in self.layers:
        output = mod(output, src)
    return output